In [1]:
import tensorflow as tf

2023-09-01 20:49:24.434002: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import autokeras as ak

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

df = pd.read_csv('data/experimental_df.csv')


In [4]:
"""X_train = df.drop(columns=['Winner'])
X_train = X_train.drop(columns=['Unnamed: 0'])
y_train = df['Winner'].astype(int)"""

"X_train = df.drop(columns=['Winner'])\nX_train = X_train.drop(columns=['Unnamed: 0'])\ny_train = df['Winner'].astype(int)"

In [5]:
X_train = df.drop(columns=['Winner'])
X_train = X_train.drop(columns=['Unnamed: 0'])
y_train = df['Winner'].astype(int)

In [6]:
X_train.columns

Index(['a', 'b', 'c', 'd', 'e', 'f', 'g'], dtype='object')

In [7]:
clf = ak.StructuredDataClassifier(overwrite=True, max_trials=30, metrics='accuracy')

clf.fit(X_train, y_train, validation_split=0.15, epochs=30)
#clf.fit(X_train, y_train, epochs=25)

Trial 30 Complete [00h 00m 02s]
val_accuracy: 0.550000011920929

Best val_accuracy So Far: 0.6499999761581421
Total elapsed time: 00h 01m 31s
INFO:tensorflow:Oracle triggered exit
Epoch 1/30
11/11 [==============================] - 0s 1ms/step - loss: 1.0788 - accuracy: 0.4441
Epoch 2/30
11/11 [==============================] - 0s 1ms/step - loss: 1.0729 - accuracy: 0.4353
Epoch 3/30
11/11 [==============================] - 0s 1ms/step - loss: 1.0850 - accuracy: 0.4324
Epoch 4/30
11/11 [==============================] - 0s 1ms/step - loss: 1.0606 - accuracy: 0.4412
Epoch 5/30
11/11 [==============================] - 0s 1ms/step - loss: 1.0751 - accuracy: 0.4382
Epoch 6/30
11/11 [==============================] - 0s 1ms/step - loss: 1.0663 - accuracy: 0.4412
Epoch 7/30
11/11 [==============================] - 0s 1ms/step - loss: 1.0715 - accuracy: 0.4559
Epoch 8/30
11/11 [==============================] - 0s 1ms/step - loss: 1.0546 - accuracy: 0.4529
Epoch 9/30
11/11 [==================

custom_f1 from https://neptune.ai/blog/implementing-the-macro-f1-score-in-keras

In [8]:
from tensorflow.keras import backend as K

def custom_f1(y_true, y_pred):
    def recall_m(y_true, y_pred):
        TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        Positives = K.sum(K.round(K.clip(y_true, 0, 1)))

        recall = TP / (Positives+K.epsilon())
        return recall


    def precision_m(y_true, y_pred):
        TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        Pred_Positives = K.sum(K.round(K.clip(y_pred, 0, 1)))

        precision = TP / (Pred_Positives+K.epsilon())
        return precision

    precision, recall = precision_m(y_true, y_pred), recall_m(y_true, y_pred)

    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [9]:
mmm = ak.StructuredDataClassifier(overwrite=True, max_trials=30, metrics=[custom_f1, 'accuracy'])

mmm.fit(X_train, y_train, validation_split=0.15, epochs=30)
#mmm.fit(X_train, y_train, epochs=25)

Trial 30 Complete [00h 00m 02s]
val_accuracy: 0.44999998807907104

Best val_accuracy So Far: 0.75
Total elapsed time: 00h 01m 30s
INFO:tensorflow:Oracle triggered exit
Epoch 1/30
11/11 [==============================] - 1s 2ms/step - loss: 1.2248 - custom_f1: 0.1380 - accuracy: 0.3176
Epoch 2/30
11/11 [==============================] - 0s 2ms/step - loss: 1.1152 - custom_f1: 0.1699 - accuracy: 0.3794
Epoch 3/30
11/11 [==============================] - 0s 2ms/step - loss: 1.0711 - custom_f1: 0.1527 - accuracy: 0.4353
Epoch 4/30
11/11 [==============================] - 0s 2ms/step - loss: 1.0448 - custom_f1: 0.1466 - accuracy: 0.4500
Epoch 5/30
11/11 [==============================] - 0s 2ms/step - loss: 1.0257 - custom_f1: 0.1758 - accuracy: 0.4735
Epoch 6/30
11/11 [==============================] - 0s 2ms/step - loss: 1.0103 - custom_f1: 0.1902 - accuracy: 0.5118
Epoch 7/30
11/11 [==============================] - 0s 2ms/step - loss: 0.9984 - custom_f1: 0.2201 - accuracy: 0.5206
Epoch 

In [10]:
import keras
keras.utils.get_custom_objects()['custom_f1'] = custom_f1

In [11]:
model = mmm.export_model()

In [12]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 7)]               0         
                                                                 
 multi_category_encoding (Mu  (None, 7)                0         
 ltiCategoryEncoding)                                            
                                                                 
 dense (Dense)               (None, 32)                256       
                                                                 
 batch_normalization (BatchN  (None, 32)               128       
 ormalization)                                                   
                                                                 
 re_lu (ReLU)                (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 32)                1056  

In [13]:
model.save('autokeras_model')

INFO:tensorflow:Assets written to: autokeras_model/assets
